**Кластеризация** — это задача группировки объектов на подмножества (кластеры) таким образом, чтобы объекты из одного кластера были более похожи друг на друга, чем на объекты из других кластеров, по какому-либо критерию.

# <center>K-Means</center>


In [ ]:
# # инициализируем алгоритм k-means с количеством кластеров 3
# kmeans = KMeans(n_clusters=3, n_init=10, random_state=42)

* `n_clusters` — количество кластеров;
* `n_init` — количество итераций алгоритма k-means;
* `random_state` — параметр для воспроизводимости результатов от запуска к запуску.

## <center>Mini-Batch K-means</center>

Данная вариация k-means используется, когда данных очень много. Из-за их объёма вычисление центров по всей выборке занимает много времени.

Для решения этой проблемы k-means на каждом шаге работает с небольшой подвыборкой данных. В общем случае упрощённый алгоритм должен сходиться к тому же результату, что и на полной выборке. Однако исследования показывают, что качество кластеров может ухудшаться по сравнению с классическим k-means. Обычно разница в кластеризации методом Mini-Batch K-means и классическим k-means заключается в пограничных точках близко расположенных кластеров.

In [ ]:
# # два кластера и подвыборки объёма 6
# kmeans = MiniBatchKMeans(n_clusters=2,random_state=42,batch_size=6)

## <center>K-means++</center>

Данную вариацию k-means используют, если признаков очень много.

Результат и время работы алгоритма зависят от изначального выбора центроидов. Чтобы минимизировать затраты, будем действовать следующим образом:

1) Первый центроид выбираем случайным образом.
2) Для каждой точки вычисляем квадрат расстояния до ближайшего центроида из тех, что уже поставлены.
3) Далее из этих точек выбираем следующий центроид так, чтобы вероятность выбора точки была пропорциональна вычисленному для неё квадрату расстояния.
4) Когда все точки выбраны, реализуем k-means.

По умолчанию при запуске k-means в sklearn используется именно алгоритм k-means++. Выбор алгоритма задаётся через параметр `init`:

* `init='random'` — для классической версии k-means;
* `init='k-means++'` — для вариации k-means++.

> Остался важный вопрос: как определить количество кластеров?

**Инерция** - формула суммы квадратов всех расстояний от точек $x_i$ до центров кластеров $C_k$, к которым принадлежат данные точки.

$$J(C)=\sum_{k=1}^{K} \sum_{i \in C_{k}}\left\|x_{i}-\mu_{k}\right\|^{2} \rightarrow \min _{C}$$

> При реализации алгоритма k-means для получения значения инерции используется атрибут `inertia_`.

In [ ]:
# def get_inertia(cluster_num, X):
# # инициализируем алгоритм кластеризации
#     k_means =  KMeans(n_clusters=cluster_num, random_state=42)
# # запускаем алгоритм k-means
#     k_means.fit(X)
# # находим значение инерции
#     inertia = k_means.inertia_
# # возвращаем значение инерции
#     return inertia

Разумеется, чем меньше эта величина, тем лучше. Однако здесь есть большая проблема: минимальное значение этой функции будет достигаться тогда, когда количество кластеров будет равняться количеству объектов (т. е. каждый кластер будет состоять из одной точки и расстояния будут нулевыми). Это уже будет ситуация переобучения, так как алгоритм чересчур сильно подстроится под данные.

Для решения этой проблемы была выведена следующая эвристика: берётся такое число кластеров, начиная с которого значение функционала $J(C)$ уменьшается уже не так быстро. Формально это можно записать следующим образом:

$$D(k)=\frac{\left|J\left(C_{k}\right)-J\left(C_{k+1}\right)\right|}{\left|J\left(C_{k-1}\right)-J\left(C_{k}\right)\right|} \rightarrow \min _{k}$$

Визуально это можно представить так:

![image.png](https://lms.skillfactory.ru/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block@MATHML_md10_2_11.png)

Можно увидеть, что инерция очень сильно уменьшается при увеличении числа кластеров с 1 до 2 и с 2 до 3 и уже не так значительно — при изменении $k$ с 3 до 4. То есть перегиб здесь находится в точке 3, и это значит, что три кластера — оптимальный вариант.

> Если в ходе решения задачи вы встречаете график, на котором невозможно найти «локоть», на помощь придёт **коэффициент силуэта**.

Для того чтобы его вычислить, используется следующая формула:

$$s_i=\frac{(b_i-a_i)}{max(a_i,b_i)}$$

* $a_i$ — среднее расстояние от данного объекта $x_i$ до объектов из того же кластера;
* $b_i$ — среднее расстояние от данного объекта $x_i$ до объектов из другого ближайшего кластера.

Для вычисления коэффициента силуэта используется `silhouette_score`.

In [ ]:
# def get_silhouette(cluster_num, X):
#     k_means =  KMeans(n_clusters=cluster_num, random_state=42)
#     k_means.fit(X)
# # подсчитаем метрику силуэта, передав данные и то, к каким кластерам относятся объекты
#     silhouette = silhouette_score(X, k_means.labels_)
#     return silhouette

# silhouette = []
# for clust_num in range(2, 10):
#     silhouette.append(get_silhouette(clust_num, X))

<u>Коэффициент силуэта обладает следующими свойствами:</u>

* Значение коэффициента находится в диапазоне от -1 до +1, где высокое значение указывает, что объект хорошо согласуется с кластером, которому он принадлежит, и плохо согласуется с «чужими» кластерами.

* Если у подавляющего большинства объектов этот коэффициент высокий, то можно считать кластеризацию достаточно качественной.

* Если же у большого числа объектов низкий или отрицательный коэффициент силуэта, то, возможно, кластеров слишком много/мало или данные просто плохо поддаются разделению на кластеры.

Для получения итогового значения рассчитывается среднее значение силуэта для всего датасета. Соответственно, для определения оптимального количества кластеров мы будем искать самую высокую точку на графике коэффициента силуэта (ведь чем больше коэффициент, тем лучше).

![image.png](https://lms.skillfactory.ru/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block@MATHML_md10_2_12.png)

# <center>EM-алгоритм</center>

Когда данные распределены в форме вытянутых эллипсов, k-means не справляется с кластеризацией:

![image.png](https://lms-cdn.skillfactory.ru/assets/courseware/v1/72322fbdd9951ad77a85c07760313285/asset-v1:Skillfactory+DSMED+2023+type@asset+block/MATHML_md10_3_1.png)

В таком случае в качестве альтернативы можно взять один из алгоритмов кластеризации EM (Expectation-maximization) — **модель гауссовой смеси (Gaussian Mixture Model, GMM)**, в которой данные описываются законом нормального распределения.

Возьмём два случайно сгенерированных распределения Гаусса (изображены на рисунке ниже синим и жёлтым цветом).

![image.png](https://lms-cdn.skillfactory.ru/assets/courseware/v1/b41662d3d6b1a35a19cb5299d01aa168/asset-v1:Skillfactory+DSMED+2023+type@asset+block/MATHML_md10_3_3.png)

Итак, каждая точка с какими-то вероятностями принадлежит к жёлтому и синему распределению. Теперь мы сможем использовать значения этих точек для расчёта новых математических ожиданий и дисперсий и построения новых распределений. То есть, по сути, дальше мы изменим распределения так, чтобы они максимально соответствовали точкам, которые мы к ним отнесли.

![image.png](https://lms-cdn.skillfactory.ru/assets/courseware/v1/e2ef0540953f34679fb32c764e23c034/asset-v1:Skillfactory+DSMED+2023+type@asset+block/MATHML_md10_3_7.png)

Далее мы ещё раз повторяем описанную ранее процедуру и снова обновляем распределения. После ряда итераций окончательные распределения будут выглядеть так:

![image.png](https://lms-cdn.skillfactory.ru/assets/courseware/v1/0022dadcc042dbd3d42a74877d4e2226/asset-v1:Skillfactory+DSMED+2023+type@asset+block/MATHML_md10_3_8.png)

Таким образом, мы смогли кластеризовать наши точки: первые четыре относятся к жёлтому кластеру, а остальные — к синему. Так и работает EM-алгоритм кластеризации.

В более общем виде последовательность действий в EM-алгоритме можно сформулировать следующим образом:

1) Выбрать количество кластеров, которое кажется нам оптимальным для наших данных.
2) Случайным образом выбрать параметры распределений в пространстве данных.
3) Для каждой точки набора данных рассчитать вероятность принадлежности к каждому кластеру (**E-шаг (expectation)** — вычисляем ожидаемый кластер для каждого объекта.).
4) Обновить параметры распределений таким образом, чтобы максимизировать вероятность принадлежности точек, отнесённых к кластеру (**M-шаг (maximization)** — оцениваем вес (априорную вероятность) и параметры распределения для каждого кластера.).
5) Повторять шаги 3-4 фиксированное число раз либо до тех пор, пока центроиды не сойдутся.

Как мы отмечали ранее, алгоритм EM очень похож на k-means и, по сути, является его упрощённым вариантом. Однако у этих алгоритмов есть различия:

* кластеры в EM эллиптические, а в k-means — сферические;

* в EM кластеризация мягкая (определяем вероятность принадлежности объекта к клластреу), а в k-means — жёсткая (определяем конкретный кластер для объекта).

Для реализации алгоритма в *sklearn* мы используем `GaussianMixture`. Для запуска алгоритма `GaussianMixture` необходимо задать следующие основные параметры:

* `n_components` — количество кластеров;
* `random_state` — так как в алгоритме есть случайность при инициализации, то для воспроизводимости результатов от запуска к запуску следует передать какое-то число.

# <center>Практика</center>